In [1]:
import pandas as pd
import regex as re
from bs4 import BeautifulSoup
from bertopic.representation import KeyBERTInspired, PartOfSpeech, MaximalMarginalRelevance
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.dimensionality import BaseDimensionalityReduction
from bertopic.vectorizers import ClassTfidfTransformer
from hdbscan import HDBSCAN
from umap import UMAP
from sentence_transformers import SentenceTransformer
import numpy as np
import warnings
import spacy
warnings.filterwarnings('ignore')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import nltk
from nltk import word_tokenize, pos_tag, ne_chunk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from bs4 import BeautifulSoup
from markdown import markdown
import re
from scipy.stats import entropy
from swifter import swifter
import matplotlib.pyplot as plt
nltk.download('wordnet')
nltk.download("stopwords")
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('maxent_ne_chunker_tab')
nltk.download('words')

random_state = 42

/home/cs/grad/opumni/Research/StudyOnBlockchainApps/.conda/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/cs/grad/opumni/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/cs/grad/opumni/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/cs/grad/opumni/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/cs/grad/opumni/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker_t

In [2]:
sw = stopwords.words('english')
stemmer = PorterStemmer()

def markdown_to_text(markdown_string):
    """ Converts a markdown string to plaintext """

    # md -> html -> text since BeautifulSoup can extract text cleanly
    html = markdown(markdown_string)

    # remove code snippets
    html = re.sub(r'<pre>(.*?)</pre>', ' ', html)
    html = re.sub(r'<code>(.*?)</code >', ' ', html)

    # extract text
    soup = BeautifulSoup(html, "html.parser")
    text = ''.join(soup.findAll(text=True))

    return text

def clean_text(text):
    text = markdown_to_text(text).strip()

    text = text.lower()

    text = re.sub(r"http\S+", " <URL> ",text) #Removing URLs 
    
    # html=re.compile(r'<.*?>') 
    
    # text = html.sub(r'',text) #Removing html tags
    
    
    emoji_pattern = re.compile("["
                               "\U0001F600-\U0001F64F"  # Emoticons
                               "\U0001F300-\U0001F5FF"  # Symbols & pictographs
                               "\U0001F680-\U0001F6FF"  # Transport & map symbols
                               "\U0001F700-\U0001F77F"  # Alchemical symbols
                               "\U0001F780-\U0001F7FF"  # Geometric shapes
                               "\U0001F800-\U0001F8FF"  # Supplemental arrows
                               "\U0001F900-\U0001F9FF"  # Supplemental symbols and pictographs
                               "\U0001FA00-\U0001FA6F"  # Symbols and pictographs extended-A
                               "\U0001FA70-\U0001FAFF"  # Symbols and pictographs extended-B
                               "\U00002702-\U000027B0"  # Dingbats
                               "\U000024C2-\U0001F251"  # Enclosed characters
                               "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text) #Removing emojis
    common_punctuation = r'.,?!:;"\'-\\/'
    text = re.sub(rf'[^\w\s{re.escape(common_punctuation)}]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def plot_cdf(values, title, xlabel, ylabel, xticks = None, yticks = None, save_to = None, ax = None):
    values = np.array(values)
    sorted_values = np.sort(values)

    cdf_R = np.arange(1, len(sorted_values) + 1) / len(sorted_values)

    if yticks is None:
        yticks = np.arange(0,1.1,0.1)

    if ax is None:
        plt.figure(figsize=(20, 6))
        plt.plot(sorted_values, cdf_R, marker='.', linestyle='none')
        plt.xlabel(xlabel)
        plt.ylabel(ylabel)
        plt.title(title)
        plt.grid(True)
        
        if xticks is not None:
            plt.xticks(xticks)
        
        if yticks is not None:
            plt.yticks(yticks)

        if save_to is not None:
            plt.savefig(save_to, format='png')
            plt.savefig(save_to, format='svg')

        plt.show()
    else:
        ax.plot(sorted_values, cdf_R, marker='.', linestyle='none')
        ax.set_xlabel(xlabel)
        ax.set_ylabel(ylabel)
        ax.set_title(title)
        ax.grid(True)
        
        if xticks is not None:
            ax.set_xticks(xticks)

        if yticks is not None:
            ax.set_yticks(yticks)

In [3]:
ISSUE_DATA_PATH_NEW = "./Data/AllIssues-New.csv"
ISSUE_DATA_PATH_OLD = "./Data/AllIssues-Old-Actives.csv"
df1 = pd.read_csv(ISSUE_DATA_PATH_NEW, low_memory=False)
df1['Body'] = df1['Body'].fillna('')

df2 = pd.read_csv(ISSUE_DATA_PATH_OLD, low_memory=False)
df2['Body'] = df2['Body'].fillna('')

df = pd.concat([df1,df2], ignore_index=True).reset_index(drop=True)

df['combinedText'] = df['Title'] + '. ' + df['Body']

In [4]:
df = df[df['combinedText'].notna()]
df = df[df['combinedText'] != ""]

df['combinedText'] = df['combinedText'].swifter.apply(clean_text)

df = df[df['combinedText'].notna()]
df = df[df['combinedText'] != ""]

df['word_count'] = df["combinedText"].apply(lambda x: len(x.split()))
df = df[df['word_count'] > 2]

df = df[df['CreatedAt'].notna()]

df.to_csv("./Data/AllIssues.csv", index=False)

Pandas Apply: 100%|██████████| 511551/511551 [11:57<00:00, 712.95it/s]  


In [2]:
df = pd.read_csv("./Data/AllIssues.csv")
data = df['combinedText'].tolist()

In [6]:
df

,Repository,IssueId,Title,Body,State,Label,CreatedAt,ClosedAt,combinedText,word_count
0,opexdev/core,516,Calculate source amount,NaN,closed,NaN,2025-03-04 23:41:59+00:00,2025-03-05 09:25:26+00:00,calculate source amount.,3
1,opexdev/core,515,Develop a service to calculate source amount i...,NaN,closed,NaN,2025-03-03 16:00:59+00:00,2025-03-03 16:14:23+00:00,develop a service to calculate source amount i...,9
2,opexdev/core,514,Pair trading availability for admin services,NaN,closed,NaN,2025-03-03 13:11:06+00:00,2025-03-04 13:28:49+00:00,pair trading availability for admin services.,6
3,opexdev/core,513,Modify webclient config,NaN,closed,NaN,2025-03-01 13:38:01+00:00,2025-03-01 13:49:11+00:00,modify webclient config.,3
4,opexdev/core,512,Organize wallet service for admin,NaN,closed,NaN,2025-03-01 11:01:23+00:00,2025-03-04 11:14:49+00:00,organize wallet service for admin.,5
...,...,...,...,...,...,...,...,...,...,...
489893,NeoNextClub/neoline,5,getAccount Alias returns undefined,getAccount alias seems like its not working an...,closed,bug,2019-05-06 04:08:18+00:00,2019-06-11 06:30:04+00:00,getaccount alias returns undefined. getaccount...,15
489894,NeoNextClub/neoline,4,Provide priority fee option when sending asset,When user use NEOLine to send assets like NEO...,closed,enhancement,2019-04-11 03:15:48+00:00,2019-05-06 14:28:08+00:00,provide priority fee option when sending asset...,44
489895,NeoNextClub/neoline,3,Provide Claim Gas function,"If user owns NEO in their wallet, GAS will be ...",closed,enhancement,2019-04-11 03:08:46+00:00,2019-04-16 11:44:09+00:00,provide claim gas function . if user owns neo ...,40
489896,NeoNextClub/neoline,2,decimal should not be allowed when transfer NEO,"Since the minimum unit of NEO is 1, decimal sh...",closed,bug,2019-04-11 03:06:52+00:00,2019-04-16 02:14:31+00:00,decimal should not be allowed when transfer ne...,25


In [7]:
embedding_model = SentenceTransformer("all-MiniLM-L6-v2") 

In [8]:
embeddings = embedding_model.encode(data, show_progress_bar=True)
np.save("./Data/Output-BERTopic/embeddings.npy", embeddings)

Batches: 100%|██████████| 15310/15310 [08:59<00:00, 28.37it/s] 


In [ ]:
embeddings = np.load("./Data/Output-BERTopic/embeddings.npy")

In [9]:
umap_model = UMAP(low_memory=False, random_state=random_state, n_components = 10, n_neighbors = 25, min_dist=0.5)
embeddings = umap_model.fit_transform(embeddings)
np.save("./Data/Output-BERTopic/reduced_embeddings.npy", embeddings)

In [ ]:
embeddings = np.load("./Data/Output-BERTopic/reduced_embeddings.npy")

In [10]:
embeddings.shape

(489898, 10)

In [11]:
umap_model_2 = UMAP(low_memory=False, random_state=random_state, n_components = 2, n_neighbors = 25, min_dist=0.5)
embeddings_2 = umap_model_2.fit_transform(np.load("./Data/Output-BERTopic/embeddings.npy"))
np.save("./Data/Output-BERTopic/reduced_embeddings_2.npy", embeddings_2)

In [12]:
# Clustering using HDBSCAN
hdbscan_model = HDBSCAN(min_cluster_size=100, min_samples=5)

ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)
vectorizer_model = CountVectorizer(stop_words= "english", ngram_range=(2, 3))
representation_model = [KeyBERTInspired(), MaximalMarginalRelevance(diversity=0.5)]
# Create BERTopic model
topic_model = BERTopic(
    #representation_model=representation_model,
    embedding_model=embedding_model,
    umap_model=BaseDimensionalityReduction(),
    hdbscan_model=hdbscan_model,
    vectorizer_model=vectorizer_model,
    calculate_probabilities=False,
    nr_topics="auto",
    verbose=True,
)
topics, probs = topic_model.fit_transform(data, embeddings=embeddings)
topic_model.save("./Data/Output-BERTopic/bertopic")

2025-03-15 23:39:48,288 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-03-15 23:39:48,289 - BERTopic - Dimensionality - Completed ✓
2025-03-15 23:39:48,321 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-03-15 23:42:58,386 - BERTopic - Cluster - Completed ✓
2025-03-15 23:42:58,387 - BERTopic - Representation - Extracting topics from clusters using representation models.
2025-03-15 23:45:22,720 - BERTopic - Representation - Completed ✓
2025-03-15 23:45:23,034 - BERTopic - Topic reduction - Reducing number of topics
2025-03-15 23:47:56,879 - BERTopic - Topic reduction - Reduced number of topics from 492 to 18
2025-03-15 23:47:58,316 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


In [3]:
topic_model = BERTopic.load("./Data/Output-BERTopic/bertopic")
topics = topic_model._map_predictions(topic_model.hdbscan_model.labels_)
probs = topic_model.hdbscan_model.probabilities_

In [14]:
topic_info = topic_model.get_topic_info()
topic_info

,Topic,Count,Name,Representation,Representative_Docs
0,-1,308460,-1_close pr_pr stop_pr stop dependabot_stop de...,"[close pr, pr stop, pr stop dependabot, stop d...",[builddeps-dev: bump types/node from 20.1.2 to...
1,0,177307,0_close pr_stop dependabot_pr stop dependabot_...,"[close pr, stop dependabot, pr stop dependabot...",[builddeps: bump ouroboros_macro from 0.9.5 to...
2,1,1252,1_v2 url compare_v2 url_url compare source_url...,"[v2 url compare, v2 url, url compare source, u...",[fixdeps: update module github.com/c-robinson/...
3,2,510,2_update dependencies chore_dependencies chore...,"[update dependencies chore, dependencies chore...","[chore: update dependencies ., chore: update d..."
4,3,401,3_new crowdin_translations crowdin_chorei18n s...,"[new crowdin, translations crowdin, chorei18n ...",[chorei18n: synchronize translations from crow...
5,4,319,4_locales en_file locales_file locales en_loca...,"[locales en, file locales, file locales en, lo...",[translate '/locales/en.json' in 'de'. the tra...
6,5,225,5_result specify_code bug feature_outcome retu...,"[result specify, code bug feature, outcome ret...",[sui code bug or feature request. steps to rep...
7,6,210,6_prs repo_pr stop dependabot_stop dependabot_...,"[prs repo, pr stop dependabot, stop dependabot...",[bump minimist from 1.2.5 to 1.2.6. bumps mini...
8,7,154,7_locale messages_locale messages xlf_messages...,"[locale messages, locale messages xlf, message...",[translate 'frontend/src/locale/messages.xlf' ...
9,8,143,8_locales en json_en json_locales en_file locales,"[locales en json, en json, locales en, file lo...",[translate '/locales/en.json' in 'ru'. the tra...


In [19]:
threshold = 0.9
new_topics = topics
print(new_topics.count(-1))

threshold=0.95
if new_topics.count(-1) > 0:
    new_topics = topic_model.reduce_outliers(documents=data, topics=new_topics, strategy="embeddings", embeddings=embeddings, threshold=threshold)
    print(new_topics.count(-1))

threshold=0.75
if new_topics.count(-1) > 0:
    new_topics = topic_model.reduce_outliers(data, new_topics, strategy="c-tf-idf", threshold=threshold)
    print(new_topics.count(-1))

threshold=0.6
if new_topics.count(-1) > 0:
    new_topics = topic_model.reduce_outliers(data, new_topics, probabilities=probs, strategy="probabilities", threshold=threshold)
    print(new_topics.count(-1))

threshold=0.5
if new_topics.count(-1) > 0:
    new_topics = topic_model.reduce_outliers(data, new_topics, strategy="distributions", threshold=threshold)
    print(new_topics.count(-1))

308460
1594
1365
1365


100%|██████████| 2/2 [00:02<00:00,  1.39s/it]

1349


In [20]:
topic_model.update_topics(data, topics=new_topics)
topic_model.save("./Data/Output-BERTopic/bertopic_!outlier")

2025-03-16 00:02:15,744 - BERTopic - WARNING: Using a custom list of topic assignments may lead to errors if topic reduction techniques are used afterwards. Make sure that manually assigning topics is the last step in the pipeline.Note that topic embeddings will also be created through weightedc-TF-IDF embeddings instead of centroid embeddings.
2025-03-16 00:02:44,767 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


In [21]:
topic_info = topic_model.get_topic_info()
topic_info

,Topic,Count,Name,Representation,Representative_Docs
0,-1,1349,-1_locales_json_en_expressjs,"[locales, json, en, expressjs, deps, dependabo...",[builddeps-dev: bump types/node from 20.1.2 to...
1,0,481426,0_the_to_this_and,"[the, to, this, and, for, in, pr, dependabot, ...",[builddeps: bump ouroboros_macro from 0.9.5 to...
2,1,1381,1_uint128_iplib_v2_this,"[uint128, iplib, v2, this, net6, functions, ur...",[fixdeps: update module github.com/c-robinson/...
3,2,783,2_ckb_ccc_alpha_dependencies,"[ckb, ccc, alpha, dependencies, chore, update,...","[chore: update dependencies ., chore: update d..."
4,3,482,3_crowdin_translations_chorei18n_synchronize,"[crowdin, translations, chorei18n, synchronize...",[chorei18n: synchronize translations from crow...
5,4,637,4_locales_en_json_transifex,"[locales, en, json, transifex, slug, file, tra...",[translate '/locales/en.json' in 'de'. the tra...
6,5,241,5_outcome_expected_steps_specify,"[outcome, expected, steps, specify, actual, ca...",[sui code bug or feature request. steps to rep...
7,6,236,6_dependabot_this_will_pr,"[dependabot, this, will, pr, it, you, merge, r...",[bump minimist from 1.2.5 to 1.2.6. bumps mini...
8,7,212,7_xlf_locale_nb_frontend,"[xlf, locale, nb, frontend, messages, transife...",[translate 'frontend/src/locale/messages.xlf' ...
9,8,143,8_ru_locales_transifex_en,"[ru, locales, transifex, en, json, slug, file,...",[translate '/locales/en.json' in 'ru'. the tra...


In [6]:
ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)
vectorizer_model = CountVectorizer(stop_words= "english", ngram_range=(1, 3))
representation_model = [KeyBERTInspired(), MaximalMarginalRelevance(diversity=0.5)]
topic_model.update_topics(data, vectorizer_model=vectorizer_model, ctfidf_model=ctfidf_model, representation_model=representation_model)

In [7]:
topic_info = topic_model.get_topic_info()
topic_info

,Topic,Count,Name,Representation,Representative_Docs
0,-1,1349,-1_sourced express_ulisesgascon expressjs_expr...,"[sourced express, ulisesgascon expressjs, expr...",[builddeps-dev: bump types/node from 20.1.2 to...
1,0,481426,0_pr upgrade dependabot_dependabot rebase_upgr...,"[pr upgrade dependabot, dependabot rebase, upg...",[builddeps: bump ouroboros_macro from 0.9.5 to...
2,1,1381,1_iplib v2_iplib v1_iplib_robinson iplib v2,"[iplib v2, iplib v1, iplib, robinson iplib v2,...",[fixdeps: update module github.com/c-robinson/...
3,2,783,2_semantic release_patch changes updated_updat...,"[semantic release, patch changes updated, upda...","[chore: update dependencies ., chore: update d..."
4,3,482,3_locale messages xlf_xlf fa translations_loca...,"[locale messages xlf, xlf fa translations, loc...",[chorei18n: synchronize translations from crow...
5,4,637,4_translation_completed_updated translate loca...,[translation_completed_updated translate local...,[translate '/locales/en.json' in 'de'. the tra...
6,5,241,5_sui code bug_sui code_compiler sui code_repr...,"[sui code bug, sui code, compiler sui code, re...",[sui code bug or feature request. steps to rep...
7,6,236,6_pr upgrade dependabot_bumps minimist commits...,"[pr upgrade dependabot, bumps minimist commits...",[bump minimist from 1.2.5 to 1.2.6. bumps mini...
8,7,212,7_messages xlf translation_xlf nb translations...,"[messages xlf translation, xlf nb translations...",[translate 'frontend/src/locale/messages.xlf' ...
9,8,143,8_json ru translations_file locales ru_ru json...,"[json ru translations, file locales ru, ru jso...",[translate '/locales/en.json' in 'ru'. the tra...


In [8]:
import openai
from bertopic.representation import OpenAI
client = openai.OpenAI(api_key="sk-...")
representation_model = OpenAI(client, model="gpt-4o-mini", chat=True)
topic_model.update_topics(data, representation_model=representation_model)

100%|██████████| 18/18 [00:07<00:00,  2.45it/s]


In [16]:

topic_model.save("./Data/Output-BERTopic/bertopic_!outlier_withRepresentation")

2025-03-16 00:38:55,392 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


TypeError: cannot pickle '_thread.RLock' object

In [32]:
topic_info = topic_model.get_topic_info()
topic_info

,Topic,Count,Name,Representation,Representative_Docs
0,-1,1349,-1_expressjs-dependency-update-4.19.2,[expressjs-dependency-update-4.19.2],[builddeps-dev: bump types/node from 20.1.2 to...
1,0,481426,0_Dependabot Version Bumps and Commands,[Dependabot Version Bumps and Commands],[builddeps: bump ouroboros_macro from 0.9.5 to...
2,1,1381,1_update iplib to v2.0.5 with performance impr...,[update iplib to v2.0.5 with performance impro...,[fixdeps: update module github.com/c-robinson/...
3,2,783,2_ckb-ccc alpha version bump and dependency up...,[ckb-ccc alpha version bump and dependency upd...,"[chore: update dependencies ., chore: update d..."
4,3,482,3_Crowdin Translation Synchronization,[Crowdin Translation Synchronization],[chorei18n: synchronize translations from crow...
5,4,637,4_Translation Completion for Locales in Blixt ...,[Translation Completion for Locales in Blixt W...,[translate '/locales/en.json' in 'de'. the tra...
6,5,241,5_SUI Code Bug Reporting Steps,[SUI Code Bug Reporting Steps],[sui code bug or feature request. steps to rep...
7,6,236,6_Dependabot Update for Miniminst Security Patch,[Dependabot Update for Miniminst Security Patch],[bump minimist from 1.2.5 to 1.2.6. bumps mini...
8,7,212,7_XLF Translation Completion for Norwegian (nb...,[XLF Translation Completion for Norwegian (nb)...,[translate 'frontend/src/locale/messages.xlf' ...
9,8,143,8_Translation Completion for Blixt Wallet Loca...,[Translation Completion for Blixt Wallet Local...,[translate '/locales/en.json' in 'ru'. the tra...


In [17]:
hierarchical_topics = topic_model.hierarchical_topics(data)
hierarchical_topics

100%|██████████| 16/16 [00:10<00:00,  1.46it/s]


,Parent_ID,Parent_Name,Topics,Child_Left_ID,Child_Left_Name,Child_Right_ID,Child_Right_Name,Distance
15,32,Dependabot PR Updates and Commands,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",30,Dependabot version bumps and commands for merg...,31,Transifex Translation Completion for Norwegian...,1.902333
14,31,Transifex Translation Completion for Norwegian...,"[3, 4, 7, 8, 9, 12]",20,Crowdin Translation Synchronization,27,Transifex Localization Updates for Norwegian T...,1.179182
13,30,Dependabot version bumps and commands for merg...,"[0, 1, 2, 5, 6, 10, 11, 13, 14, 15, 16]",29,update github.com/c-robinson/iplib to v2.0.5 w...,26,Dependabot Dependency Updates,1.139138
12,29,update github.com/c-robinson/iplib to v2.0.5 w...,"[1, 5, 10, 15, 16]",28,sourcery pull request refactorings and merging...,23,update-github.com/c-robinson/iplib-to-v2.0.5,1.132396
11,28,sourcery pull request refactorings and merging...,"[5, 15]",15,sourcery refactor pull requests,5,SUI Code Bug Reporting Steps,0.951921
10,27,Transifex Localization Updates for Norwegian T...,"[4, 7, 8, 12]",18,Translation of XLF messages in Norwegian (nb) ...,19,Translation Updates for Blixt Wallet Locales,0.881345
9,26,Dependabot Dependency Updates,"[0, 2, 6, 11, 13, 14]",25,Dependabot Dependency Updates,2,ckb-ccc alpha version bump and dependency updates,0.837605
8,25,Dependabot Dependency Updates,"[0, 6, 11, 13, 14]",13,bump ethersphere/bee versions,24,Dependabot Version Bumps and Commands,0.805685
7,24,Dependabot Version Bumps and Commands,"[0, 6, 11, 14]",21,Dependabot Version Bumps and Commands,11,Bumping Defichain/Jellyfish Dependencies to La...,0.711233
6,23,update-github.com/c-robinson/iplib-to-v2.0.5,"[1, 10, 16]",22,Update iplib to v2 with uint128 enhancements,10,golang-lru v2.0.7 release updates and contribu...,0.595912


In [ ]:
plot_cdf(hierarchical_topics["Distance"], "CDF", "count", "CDF", xticks=np.arange(0,max(hierarchical_topics["Distance"]),0.1))

In [ ]:
# topics_to_merge = []
# merged = set()
# for topics, distance in zip(hierarchical_topics["Topics"].to_list(),hierarchical_topics["Distance"].to_list()):
#     if len(topics) <= 2 and not set(topics) <= merged and distance <= 0.5:
#         topics_to_merge.append(topics)
#         for t in topics:
#             merged.add(t)
# topic_model.merge_topics(data, topics_to_merge)

In [18]:
topic_model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)

In [19]:
topic_model.visualize_topics()

In [20]:
topic_model.visualize_heatmap()

In [21]:
topic_model.visualize_barchart()

In [35]:
topics = topic_model.get_document_info(data)["Topic"]

In [36]:
df["Topic"] = topics

In [38]:
unique_values = df["Topic"].unique()
for value in unique_values:
    df_subset = df[df["Topic"] == value]
    filename = f"./Data/Output-BERTopic/Topics/topic_{value}.csv" 
    df_subset.to_csv(filename, index=False)
    print(f"Saved: {filename}")

Saved: ./Data/Output-BERTopic/Topics/topic_0.csv
Saved: ./Data/Output-BERTopic/Topics/topic_4.csv
Saved: ./Data/Output-BERTopic/Topics/topic_6.csv
Saved: ./Data/Output-BERTopic/Topics/topic_14.csv
Saved: ./Data/Output-BERTopic/Topics/topic_1.csv
Saved: ./Data/Output-BERTopic/Topics/topic_-1.csv
Saved: ./Data/Output-BERTopic/Topics/topic_2.csv
Saved: ./Data/Output-BERTopic/Topics/topic_3.csv
Saved: ./Data/Output-BERTopic/Topics/topic_11.csv
Saved: ./Data/Output-BERTopic/Topics/topic_13.csv
Saved: ./Data/Output-BERTopic/Topics/topic_16.csv
Saved: ./Data/Output-BERTopic/Topics/topic_9.csv
Saved: ./Data/Output-BERTopic/Topics/topic_10.csv
Saved: ./Data/Output-BERTopic/Topics/topic_5.csv
Saved: ./Data/Output-BERTopic/Topics/topic_7.csv
Saved: ./Data/Output-BERTopic/Topics/topic_12.csv
Saved: ./Data/Output-BERTopic/Topics/topic_15.csv
Saved: ./Data/Output-BERTopic/Topics/topic_8.csv
